In [1]:
from utils import load_data, accuracy
import numpy as np
import  torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.optim as optim
import time

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
A, X, y, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [5]:
A.shape

torch.Size([2708, 2708])

In [6]:
idx_val.shape

torch.Size([300])

In [7]:
idx_train

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139])

In [8]:
class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter("bias", None)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1 / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
            
    def forward(self, input_, adj):
        support = torch.mm(input_, self.weight) #mm: 行列積
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output
    
    def __repr__(self):
        return self.__class__.__name__ + "(" \
                         + str(self.in_features) + " -> " \
                        + str(self.out_features) + ")"

In [9]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dr_rate):
        super().__init__()
        
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dr_rate = dr_rate
        
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dr_rate, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [10]:
fastmode = False
seed = 42
epochs = 200
lr = 0.01
weight_decay= 5e-4
hidden = 16
dr_rate = 0.5
n_class = len(np.unique(y.numpy()))

In [11]:
model = GCN(nfeat=X.shape[1], nhid=hidden, nclass=n_class, dr_rate=dr_rate)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [12]:
model = model.to(device)
X = X.to(device)
A = A.to(device)
y = y.to(device)
idx_train = idx_train.to(device)
idx_val = idx_val.to(device)
idx_test = idx_test.to(device)

In [13]:
def train(epoch):
    t = time.time()
    
    model.train() #train modeにしている
    optimizer.zero_grad()
    output = model(X, A)
    loss_train = F.nll_loss(output[idx_train], y[idx_train])
    acc_train = accuracy(output[idx_train], y[idx_train])
    loss_train.backward()
    optimizer.step()
    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(X, A)

    loss_val = F.nll_loss(output[idx_val], y[idx_val])
    acc_val = accuracy(output[idx_val], y[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [14]:
def test():
    model.eval()
    output = model(X, A)
    loss_test = F.nll_loss(output[idx_test], y[idx_test])
    acc_test = accuracy(output[idx_test], y[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [15]:
t_total = time.time()
for epoch in range(epochs):
    train(epoch)

Epoch: 0001 loss_train: 1.9121 acc_train: 0.3000 loss_val: 1.8914 acc_val: 0.3500 time: 0.0286s
Epoch: 0002 loss_train: 1.9024 acc_train: 0.2929 loss_val: 1.8834 acc_val: 0.3500 time: 0.0189s
Epoch: 0003 loss_train: 1.8964 acc_train: 0.2929 loss_val: 1.8766 acc_val: 0.3500 time: 0.0187s
Epoch: 0004 loss_train: 1.8854 acc_train: 0.2929 loss_val: 1.8698 acc_val: 0.3500 time: 0.0190s
Epoch: 0005 loss_train: 1.8839 acc_train: 0.2929 loss_val: 1.8635 acc_val: 0.3500 time: 0.0187s
Epoch: 0006 loss_train: 1.8728 acc_train: 0.2929 loss_val: 1.8573 acc_val: 0.3500 time: 0.0186s
Epoch: 0007 loss_train: 1.8618 acc_train: 0.2929 loss_val: 1.8511 acc_val: 0.3500 time: 0.0187s
Epoch: 0008 loss_train: 1.8628 acc_train: 0.2929 loss_val: 1.8448 acc_val: 0.3500 time: 0.0184s
Epoch: 0009 loss_train: 1.8561 acc_train: 0.2929 loss_val: 1.8386 acc_val: 0.3500 time: 0.0189s
Epoch: 0010 loss_train: 1.8403 acc_train: 0.2929 loss_val: 1.8323 acc_val: 0.3500 time: 0.0188s
Epoch: 0011 loss_train: 1.8342 acc_train

In [16]:
test()

Test set results: loss= 0.6953 accuracy= 0.8350


# torch_geometric

In [18]:
!pip install torch_geometric

  Using cached https://files.pythonhosted.org/packages/3c/fe/630bacb652680f6d481b9febbb3e2c3869194a1a5fc3401a4a41195a2f8f/rdflib-4.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/9f/b36f7774ff5ea8e428fdcfc4bb332c39ee5b9362ddd3d40d9516a55221b2/isodate-0.6.0-py2.py3-none-any.whl
conda 4.6.14 requires ruamel_yaml>=0.11.14, which is not installed.
tensorflow 1.11.0 has requirement keras-applications>=1.0.5, but you'll have keras-applications 1.0.4 which is incompatible.
tensorflow 1.11.0 has requirement keras-preprocessing>=1.0.3, but you'll have keras-preprocessing 1.0.2 which is incompatible.
tensorflow 1.11.0 has requirement setuptools<=39.1.0, but you'll have setuptools 39.2.0 which is incompatible.
tensorflow 1.11.0 has requirement tensorboard<1.12.0,>=1.11.0, but you'll have tensorboard 1.10.0 which is incompatible.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' co

In [ ]:
! pip install torch_spa

In [19]:
import torch 
import torch_geometric

ModuleNotFoundError: No module named 'torch_sparse'